In [ ]:
!pip install sentence_transformers
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.3 MB/s 
     |████████████████████████████████| 4.4 MB 16.9 MB/s 
     |████████████████████████████████| 1.2 MB 47.8 MB/s 
     |████████████████████████████████| 101 kB 13.6 MB/s 
     |████████████████████████████████| 596 kB 56.5 MB/s 
     |████████████████████████████████| 6.6 MB 27.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=94f347171717ddec3e504e61bab045d3d404bc8832f33813059f8cdecf2973d2
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-

In [ ]:
import numpy as np
import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

- 삼성전자를 지원한 이유와 입사 후 회사에서 이루고 싶은 꿈을 기술하십시오.
- 생산관리, 공정관리, 품질관리

In [ ]:
doc = """
"생산의 모든 부분을 알고 있는 Generalist" 

삼성전자에 지원을 한 이유는 제가 궁극적으로 생각했던 생산관리 직무에서의 모든 부분의 업무를 수행할 수 있고 All around player로 성장하여 전문가가 될 수 있기 때문입니다. 

대학교 2학년 때, 데이터베이스 응용 및 설계 과목을 들으면서 SQL을 통해 데이터를 추출하는 것에 가장 흥미를 느껴 A+라는 성적을 받았고, 생산관리 업무에서도 SQL을 접하여 데이터를 다루는 업무를 꿈 꿨습니다. 현재는 OOO에서 기체 부품 생산관리 업무를 통해 생산기획, 원자재 사용 최적화, 생산량 달성을 수행했습니다. 이러한 경험과 제 꿈을 합쳐 업무를 수행할 수 있는 곳이 삼성전자라고 판단이 되었습니다. 

입사 후에는 장기적으로 같은 팀의 후배들과 유관부서들의 실무자에게 존경 받고 가장 배울 점이 많은 선배로 인식되는 것이 꿈입니다. 같이 일하는 사람들로부터 인정받으면 업무에 더 강한 동기가 부여되기 때문입니다. 이를 위해서 신입사원으로 입사하면 유관부서, 상사, 선배분들의 업무 태도, 방식, 커뮤니케이션을 면밀히 관찰하여 제 것으로 만들 것입니다. 기존에 해왔던 경험이 조직에 빠르게 적응할 수 있도록 하기 때문입니다. 또한, OJT를 통해 업무 습득은 물론이고 업무 외적인 시간을 최대한 활용하여 빠르게 회사 및 직무 이해도를 넓히겠습니다.
"""

In [ ]:
okt = Okt()

tokenized_doc = okt.pos(doc)
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

print('품사 태깅 10개만 출력 :',tokenized_doc[:10])
print('명사 추출 :',tokenized_nouns)

품사 태깅 10개만 출력 : [('\n', 'Foreign'), ('"', 'Punctuation'), ('생산', 'Noun'), ('의', 'Josa'), ('모든', 'Noun'), ('부분', 'Noun'), ('을', 'Josa'), ('알', 'Noun'), ('고', 'Josa'), ('있는', 'Adjective')]
명사 추출 : 생산 모든 부분 알 전자 지원 이유 제 궁극 생각 생산 관리 직무 모든 부분 업무 수행 수 로 전문가 수 때문 대학교 학년 때 데이터베이스 응용 및 설계 과목 통해 데이터 추출 것 가장 흥미 성적 생산 관리 업무 접 데이터 업무 꿈 현재 기체 부품 생산 관리 업무 통해 생산 기획 원자재 사용 최적화 생산 량 달성 수행 경험 제 꿈 업무 수행 수 곳 전자 판단 입사 후 장기 팀 후배 유관 부서 실무 존경 가장 점 선배 인식 것 꿈 일 사람 로부터 업무 더 동기 부여 때문 위해 신입사원 입사 유관 부서 상사 선배 업무 태도 방식 커뮤니케이션 관찰 제 것 것 기존 경험 조직 수 때문 또한 를 통해 업무 습득 업무 외적인 시간 최대한 활용 회사 및 직무 이해도


여기서는 사이킷런의 CountVectorizer를 사용하여 단어를 추출합니다. CountVectorizer를 사용하는 이유는 n_gram_range의 인자를 사용하면 쉽게 n-gram을 추출할 수 있기 때문입니다. 예를 들어, (2, 3)으로 설정하면 결과 후보는 2개의 단어를 한 묶음으로 간주하는 bigram과 3개의 단어를 한 묶음으로 간주하는 trigram을 추출합니다.

In [ ]:
n_gram_range = (2, 3)

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5])

trigram 개수 : 186
trigram 다섯개만 출력 : ['가장 선배' '가장 선배 인식' '가장 흥미' '가장 흥미 성적' '경험 업무']


다음으로 이제 문서와 문서로부터 추출한 키워드들을 SBERT를 통해서 수치화 할 차례입니다. 한국어를 포함하고 있는 다국어 SBERT를 로드합니다.

In [ ]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

이제 문서와 가장 유사한 키워드들을 추출합니다. 여기서는 문서와 가장 유사한 키워드들은 문서를 대표하기 위한 좋은 키워드라고 가정합니다. 상위 5개의 키워드를 출력합니다.

In [ ]:
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

['업무 데이터 업무', '업무 데이터', '학년 데이터베이스 응용', '데이터베이스 응용', '데이터베이스 응용 설계']


다양한 키워드를 얻기 위한 방법


*   Max Sum Similarity
*   Maximal Marginal Relevance

# Max Sum Similarity
데이터 쌍 사이의 최대 합 거리는 데이터 쌍 간의 거리가 최대화되는 데이터 쌍으로 정의됩니다. 여기서의 의도는 후보 간의 유사성을 최소화하면서 문서와의 후보 유사성을 극대화하고자 하는 것입니다.
이를 위해 상위 10개의 키워드를 선택하고 이 10개 중에서 서로 가장 유사성이 낮은 5개를 선택합니다. 낮은 nr_candidates를 설정하면 결과는 출력된 키워드 5개는 기존의 코사인 유사도만 사용한 것과 매우 유사한 것으로 보입니다.

In [ ]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [ ]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)

['로부터 업무 동기', '통해 업무 습득', '학년 데이터베이스 응용', '데이터베이스 응용', '데이터베이스 응용 설계']

상대적으로 높은 nr_candidates는 더 다양한 키워드 5개를 만듭니다.

In [ ]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=50)

['궁극 생각 생산', '대학교 학년 데이터베이스', '전자 지원 이유', '업무 태도 방식', '데이터베이스 응용']

# Maximal Marginal Relevance
텍스트 요약 작업에서 중복을 최소화하고 결과의 다양성을 극대화하기 위해 노력합니다. 먼저 문서와 가장 유사한 키워드/키프레이즈를 선택합니다. 그런 다음 문서와 유사하고 이미 선택된 키워드/키프레이즈와 유사하지 않은 새로운 후보를 반복적으로 선택합니다.

In [ ]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

만약 상대적으로 낮은 diversity 값을 설정한다면, 결과는 기존의 코사인 유사도만 사용한 것과 매우 유사한 것으로 보입니다.

In [ ]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)

['데이터베이스 응용 설계', '통해 업무 습득', '학년 데이터베이스 응용', '업무 데이터', '데이터베이스 응용']

그러나 상대적으로 높은 diversity값은 다양한 키워드 5개를 만들어냅니다.

In [ ]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.7)

['데이터베이스 응용 설계', '가장 흥미 성적', '통해 업무 습득', '부서 상사 선배', '장기 후배 유관']